In [11]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import PromptTemplate


def get_weather(lon, lat):
    print(lon)
    print(lat)


function = {
    "name": "get_weather",
    "description": (
        "function that takes longitude and latitude to find the weather of a"
        " place"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "lon": {
                "type": "string",
                "description": "The longitude coordinate",
            },
            "lat": {
                "type": "string",
                "description": "The latitude coordinate",
            },
        },
    },
    "required": ["lon", "lat"],
}

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
).bind(
    functions=[function],
    function_call="auto",
)

prompt = PromptTemplate.from_template("What is the weather in {city}?")

chain = prompt | llm

response = chain.invoke({"city": "New York"})

response = response.additional_kwargs["function_call"]["arguments"]

In [12]:
import json

r = json.loads(response)

get_weather(r["lon"], r["lat"])

-74.006
40.7128
